In [10]:
!nvidia-smi

Wed Jul 19 19:55:28 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P0    28W /  70W |   4440MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla T4            Off  | 00000000:00:05.0 Off |                    0 |
| N/A   

In [1]:
!pip install torch
!pip install bitsandbytes
!pip install accelerate@git+https://github.com/huggingface/accelerate.git
!pip install transformers@git+https://github.com/huggingface/transformers.git
!pip install peft@git+https://github.com/huggingface/peft.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 18.4 MB/s eta 0:00:0000:0100:01
  Cloning https://github.com/huggingface/accelerate.git to /tmp/pip-install-n8oe9qcf/accelerate_a529b76d4806434199ce54dee8f3b2e7
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate.git /tmp/pip-install-n8oe9qcf/accelerate_a529b76d4806434199ce54dee8f3b2e7
  Resolved https://github.com/huggingface/accelerate.git to commit 505b5be044dbb8b348053758d6cddb5a2f882187
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for accelerate: filename=accelerate-0.22.0.dev0-py3-none-any.whl size=248353 sha256=92e631bc296b1200ad55562841bf5261feb72c49459c3071d61804a5d271dc69
  Stored in directory: /tmp/pip-ephem-wheel-cache-zylxoxv9/wheels/60/53/1d/f8f7d9ed24f2b70cf9b37ecd31318a274049263effcc4b5bf3
Successfully built accelerate
  Cloning https://github.com/huggingface/tran

In [12]:
# Uncomment if necessary
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
import torch
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

In [3]:
base_model_name_or_path = "VMware/open-llama-13b-open-instruct"

tokenizer = AutoTokenizer.from_pretrained(base_model_name_or_path, padding_side="right", use_fast=False)
# tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    base_model_name_or_path,
    load_in_4bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
    # load_in_8bit=True,
    # trust_remote_code=True,
    low_cpu_mem_usage=True,
)

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565, and set the legacy attribute accordingly.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
# Load the LoRA adapter

lora_model_name_or_path = "Gustrd/open-llama-13b-lora-cabra-adapter"
model = PeftModel.from_pretrained(model, lora_model_name_or_path)

model.eval()

if torch.__version__ >= "2":
    model = torch.compile(model)

In [5]:
def generate_prompt(instruction, input=None):
    if input:
        return f"""Abaixo está uma instrução que descreve uma tarefa, juntamente com uma entrada que fornece mais contexto. Escreva uma resposta que complete adequadamente o pedido.

### Instrução:
{instruction}

### Entrada:
{input}

### Resposta:
"""
    else:
        return f"""Abaixo está uma instrução que descreve uma tarefa. Escreva uma resposta que complete adequadamente o pedido.

### Instrução:
{instruction}

### Resposta:
"""

In [6]:
def instruct(
    instruction ,
    temperature=0.3,
    top_p=0.9,
    max_new_tokens=200,
    input=None,
    **kwargs,
):
    prompt = generate_prompt(instruction, input)
    input_ids = tokenizer(prompt, return_tensors="pt")["input_ids"].to(model.device)
    input_length = input_ids.shape[1]

    do_sample = temperature > 0.0
    
    generated_outputs = model.generate(
        input_ids=input_ids,
        generation_config=GenerationConfig(
            temperature=temperature,
            top_p=top_p,
            do_sample=do_sample,
            max_new_tokens=max_new_tokens,
            **kwargs,
          ),
        return_dict_in_generate=True,
    )
    generated_tokens = generated_outputs.sequences[0, input_length:]
    generated_text = tokenizer.decode(generated_tokens, skip_special_tokens=True)
    
    splitted_generated_text = generated_text.split('\n')
    first_generated_text = splitted_generated_text[0]
    
    return first_generated_text

In [32]:
traducao = instruct("Traduza o seguinte texto do inglês para o português.",input="He's never done anything.")

print(traducao)

Nunca fez nada.
Nunca fez nada.


In [7]:
import pandas as pd

#First time
#opus_100_pt_validation = pd.read_parquet('https://huggingface.co/api/datasets/opus100/parquet/en-pt/validation/0.parquet')
#Continuing
opus_100_pt_validation = pd.read_parquet('/notebooks/opus_100_pt_validation_cabra0.parquet')

In [8]:
max_string = ""

for json in opus_100_pt_validation.translation:
    
    string = json['en'] 
    if len(string) > len(max_string):
        max_string = string
        
    string = json['pt'] 
    if len(string) > len(max_string):
        max_string = string
        
print("The biggest string is: " + max_string)

tokens = tokenizer.tokenize(max_string)
num_tokens = len(tokens)

print("The maximum tokens number is: " + str(num_tokens))

The biggest string is: having regard to the Annual Report of the Court of Auditors on the implementation of the budget concerning the financial year 2011, together with the institutions’ replies [3], and to the Court of Auditors’ special reports,having regard to the statement of assurance [4] as to the reliability of the accounts and the legality and regularity of the underlying transactions provided by the Court of Auditors for the financial year 2011 pursuant to Article 287 of the Treaty on the Functioning of the European Union,having regard to the Council’s recommendation of 12 February 2013 on discharge to be given to the Commission in respect of the implementation of the budget for the financial year 2011 (05752/2013 – C7-0038/2013),
The maximum tokens number is: 181


In [9]:
dataset = opus_100_pt_validation.translation

row_number = len(dataset)
current_row = 0

for json in opus_100_pt_validation.translation:
    current_row=current_row+1
    
    if 'cabra0' in json and json['cabra0']:
        print(str(current_row)+ "/" + str(row_number)+ " - Tradução existente: " + json['cabra0'] )
    else:
        originalString = json['en'] 
        traducao = instruct("Traduza o seguinte texto do inglês para o português.",input=originalString)
        print(str(current_row)+ "/" + str(row_number)+ " - Tradução: " + traducao)
        json['cabra0']=traducao
        opus_100_pt_validation.to_parquet('/notebooks/opus_100_pt_validation_cabra0.parquet')
    

1/2000 - Tradução existente: Ele nunca fez nada.
2/2000 - Tradução existente: 50/50 shot?
3/2000 - Tradução existente: Sim, eu vou ser bom.
4/2000 - Tradução existente: Não, não foi minha contribuição.
5/2000 - Tradução existente: Eles então seguem!
6/2000 - Tradução existente: Porque você não lavou.
7/2000 - Tradução existente: Parker não tinha um telefone, e não tinha nenhuma outra maneira de comunicar-se.
8/2000 - Tradução existente: Vou te contar o que o Detective Bird aqui pense que aconteceu.
9/2000 - Tradução existente: Se você amei-me, você teria paddleado.
10/2000 - Tradução existente: - Temos dois suspeitos.
11/2000 - Tradução existente: - Abriu a porta...
12/2000 - Tradução existente: Os prêstimos podem atrair uma subsídia de interesse para a qual fornece-se uma conta operacional.
13/2000 - Tradução existente: Não um centavo. Então, Mikey Molloy pretendeu ter um de seus ataques... para que eu pudesse entrar quando ninguém estava olhando.
14/2000 - Tradução existente: Juro-vo

/usr/local/lib/python3.9/dist-packages/bitsandbytes/nn/modules.py:224: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')


1974/2000 - Tradução: Amigos nunca precisam comprar coisas de mim.
1975/2000 - Tradução: Eu olhei em redor, e eu era o único que estava a pé.
1976/2000 - Tradução: O que eu quero dizer é que, em muitos casos, o problema com o dispositivo é que os arquivos .sys não estão corretos.
1977/2000 - Tradução: As propriedades de propriedade coletiva e de gado e animais não estão bem suficientes, mas as construções para os kulaks são muito boas.
1978/2000 - Tradução: Não saber o nome do Primeiro-Ministro não é um crime capital.
1979/2000 - Tradução: Não se preocupe, não é problema.
1980/2000 - Tradução: O ponto é, eu acho, que também precisamos ser capazes de ser precisos quando se trata de pessoas ’ s saúde.
1981/2000 - Tradução: Carro de aluguel Carro de aluguel empresas podem ser encontrados em todos os grandes centros, e em aeroportos internacionais.
1982/2000 - Tradução: Sure, I'd be happy to listen to your little secret.
1983/2000 - Tradução: Aqui está o texto em inglês:
1984/2000 - Traduç

In [ ]:
opus_100_pt_validation.to_parquet('/notebooks/opus_100_pt_validation_cabra0.parquet')

In [48]:

saved_parquet = pd.read_parquet('/notebooks/opus_100_pt_validation_cabra0.parquet')
print(saved_parquet.translation[200])                        

{'cabra0': None, 'en': 'To foster ethics in business, Camargo Corrêa Group elaborated the Camargo Corrêa Business Code of Conduct, which gathers the principles guiding the relationship of companies, executives, shareholders, and employees.', 'pt': 'Por incentivar a ética na condução dos negócios, o Grupo Camargo Corrêa elaborou o Código de Conduta Empresarial Camargo Corrêa, que reúne os princípios que norteiam o relacionamento de empresas, executivos, acionistas e profissionais.'}
